<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Run Notebook on AML
This noteboook provides an example of how to directly submit notebook to AzureML compute targets. 

# 0 Setup environment

### Install azure.contrib.notebook package

In [19]:
!pip install "azure.contrib.notebook>=1.0.21.1"

  Could not find a version that satisfies the requirement azure.contrib.notebook>=1.0.21.1 (from versions: )
No matching distribution found for azure.contrib.notebook>=1.0.21.1


### Set up AzureML workspace
Please refer to [configuration.ipynb](https://github.com/Azure/MachineLearningNotebooks/blob/master/configuration.ipynb) to set up Azure account, subscription, workspace and compute target. Below code assumes config.json file is in place.

In [20]:
import os
from azureml.core import Workspace
from os import path, makedirs
import shutil
from azureml.core import Experiment
from azureml.core.runconfig import RunConfiguration
from azureml.contrib.notebook.notebook_run_config import NotebookRunConfig, PapermillExecutionHandler

subscription_id = os.getenv("SUBSCRIPTION_ID", default="<my-subscription-id>")
resource_group = os.getenv("RESOURCE_GROUP", default="<my-resource-group>")
workspace_name = os.getenv("WORKSPACE_NAME", default="<my-workspace-name>")
workspace_region = os.getenv("WORKSPACE_REGION", default="eastus2")

ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)

This example uses run-based AzureML Compute. Alternatively, you can use persistent compute target (cpu or gpu cluster) or remote virtual machines. For more details, see [How to set up training targets](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets). The VM size (`STANDARD_D2_V2`) used here costs $0.114/hour.

In [ ]:
NOTEBOOK_NAME = 'sar_movielens.ipynb'
EXPERIMENT_NAME = 'sdk-papermill'
exp_directory = 'test_dir' + EXPERIMENT_NAME
# create experiment directory
if not path.exists(exp_directory):
    makedirs(exp_directory)
notebook_path = path.join(exp_directory, NOTEBOOK_NAME)
shutil.copy(NOTEBOOK_NAME, notebook_path)
        
exp = Experiment(workspace=ws, name=EXPERIMENT_NAME)

run_config_aml = RunConfiguration()
run_config_aml.target = "amlcompute" # possible to use different compute targets
run_config_aml.amlcompute.vm_size = 'STANDARD_D2_V2'
run_config_aml.environment.docker.enabled = True
run_config_aml.environment.docker.base_image = DEFAULT_CPU_IMAGE
run_config_aml.environment.python.user_managed_dependencies = False
run_config_aml.auto_prepare_environment = True
run_config_aml.environment.python.conda_dependencies = CondaDependencies(conda_dependencies_file_path='../../myenv2.yaml')

handler = PapermillExecutionHandler(kernel_name='python3')

cfg = NotebookRunConfig(source_directory='../../',
                                notebook='notebooks/00_quick_start/' + notebook,
                                output_notebook='outputs/out.ipynb',
                                parameters={"MOVIELENS_DATA_SIZE": "100k", "TOP_K": 10},
                                run_config=run_config_aml)
run = exp.submit(cfg)
run

You should see simialar output as below
![Experiment submit output](https://docs.microsoft.com/en-us/azure/machine-learning/service/media/quickstart-get-started/view_exp.png)
Azure Portal looks like this
![Azure Portal Experiment](https://docs.microsoft.com/en-us/azure/machine-learning/service/media/quickstart-get-started/web-results.png)

In [22]:
run.wait_for_completion(show_output=True)

RunId: sdk-papermill_1553581647_9f0859a8

Streaming azureml-logs/20_image_build_log.txt

2019/03/26 06:27:52 Using acb_vol_44c02fa8-b94f-486d-b2a8-1faec2149c6a as the home volume
2019/03/26 06:27:52 Creating Docker network: acb_default_network, driver: 'bridge'
2019/03/26 06:27:52 Successfully set up Docker network: acb_default_network
2019/03/26 06:27:52 Setting up Docker configuration...
2019/03/26 06:27:53 Successfully set up Docker configuration
2019/03/26 06:27:53 Logging in to registry: setupwsacrjscpbzxn.azurecr.io
2019/03/26 06:27:54 Successfully logged into setupwsacrjscpbzxn.azurecr.io
2019/03/26 06:27:54 Executing step ID: acb_step_0. Working directory: '', Network: 'acb_default_network'
2019/03/26 06:27:54 Obtaining source code and scanning for dependencies...
2019/03/26 06:27:54 Successfully obtained source code and scanned for dependencies
2019/03/26 06:27:54 Launching container with name: acb_step_0
Sending build context to Docker daemon  45.06kB

Step 1/15 : FROM mcr.mi


pysocks-1.6.8        | 22 KB     |            |   0% 
pysocks-1.6.8        | 22 KB     | ########## | 100% 

intel-openmp-2019.3  | 886 KB    |            |   0% 
intel-openmp-2019.3  | 886 KB    | ########7  |  87% 
intel-openmp-2019.3  | 886 KB    | ########## | 100% 

pip-19.0.3           | 1.8 MB    |            |   0% 
pip-19.0.3           | 1.8 MB    | #######7   |  77% 
pip-19.0.3           | 1.8 MB    | #########2 |  93% 
pip-19.0.3           | 1.8 MB    | ########## | 100% 

xorg-libxdmcp-1.1.3  | 18 KB     |            |   0% 
xorg-libxdmcp-1.1.3  | 18 KB     | ########## | 100% 

preshed-2.0.1        | 84 KB     |            |   0% 
preshed-2.0.1        | 84 KB     | ########## | 100% 

jinja2-2.10          | 89 KB     |            |   0% 
jinja2-2.10          | 89 KB     | ########## | 100% 

jpeg-9c              | 251 KB    |            |   0% 
jpeg-9c              | 251 KB    | ########## | 100% 

ptyprocess-0.6.0     | 22 KB     |            |   0% 
ptyprocess-0.6.0    


scipy-1.2.1          | 18.1 MB   |            |   0% 
scipy-1.2.1          | 18.1 MB   | ##7        |  28% 
scipy-1.2.1          | 18.1 MB   | ######9    |  70% 
scipy-1.2.1          | 18.1 MB   | ########8  |  88% 
scipy-1.2.1          | 18.1 MB   | ########## | 100% 

setuptools-40.8.0    | 626 KB    |            |   0% 
setuptools-40.8.0    | 626 KB    | ########5  |  85% 
setuptools-40.8.0    | 626 KB    | ########## | 100% 

pexpect-4.6.0        | 75 KB     |            |   0% 
pexpect-4.6.0        | 75 KB     | ########## | 100% 

fontconfig-2.12.1    | 938 KB    |            |   0% 
fontconfig-2.12.1    | 938 KB    | ########5  |  85% 
fontconfig-2.12.1    | 938 KB    | ########## | 100% 

wheel-0.33.1         | 34 KB     |            |   0% 
wheel-0.33.1         | 34 KB     | ########## | 100% 

blas-1.1             | 1 KB      |            |   0% 
blas-1.1             | 1 KB      | ########## | 100% 

ninja-1.9.0          | 1.6 MB    |            |   0% 
ninja-1.9.0          


pytorch-cpu-1.0.1    | 26.8 MB   |            |   0% 
pytorch-cpu-1.0.1    | 26.8 MB   |            |   0% 
pytorch-cpu-1.0.1    | 26.8 MB   | 6          |   6% 
pytorch-cpu-1.0.1    | 26.8 MB   | ##9        |  29% 
pytorch-cpu-1.0.1    | 26.8 MB   | ####5      |  46% 
pytorch-cpu-1.0.1    | 26.8 MB   | ######5    |  65% 
pytorch-cpu-1.0.1    | 26.8 MB   | #######8   |  78% 
pytorch-cpu-1.0.1    | 26.8 MB   | ########7  |  88% 
pytorch-cpu-1.0.1    | 26.8 MB   | #########4 |  95% 
pytorch-cpu-1.0.1    | 26.8 MB   | ########## | 100% 

requests-2.21.0      | 84 KB     |            |   0% 
requests-2.21.0      | 84 KB     | ########## | 100% 

cryptography-2.5     | 645 KB    |            |   0% 
cryptography-2.5     | 645 KB    | ########   |  81% 
cryptography-2.5     | 645 KB    | #########7 |  98% 
cryptography-2.5     | 645 KB    | ########## | 100% 

attrs-19.1.0         | 32 KB     |            |   0% 
attrs-19.1.0         | 32 KB     | ########## | 100% 

pyrsistent-0.14.11   | 

  Stored in directory: /root/.cache/pip/wheels/45/cb/7e/ce6e6062c69446e39e328170524ca8213498bc66a74c6a210b
Successfully built pathspec



#
# To activate this environment, use:
# > source activate /azureml-envs/azureml_5a000b1ed9d26a229bff8d937136f609
#
# To deactivate an active environment, use:
# > source deactivate
#

Removing intermediate container acb1559ee2cc
 ---> 6e0b169670f0
Step 9/15 : ENV PATH /azureml-envs/azureml_5a000b1ed9d26a229bff8d937136f609/bin:$PATH
 ---> Running in b5e661019f52
Removing intermediate container b5e661019f52
 ---> c1b6bffa92c6
Step 10/15 : ENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/azureml_5a000b1ed9d26a229bff8d937136f609
 ---> Running in a6baf120ad8c
Removing intermediate container a6baf120ad8c
 ---> 79b8e98b2a87
Step 11/15 : ENV LD_LIBRARY_PATH /azureml-envs/azureml_5a000b1ed9d26a229bff8d937136f609/lib:$LD_LIBRARY_PATH
 ---> Running in 49c76be0f9df
Removing intermediate container 49c76be0f9df
 ---> 4beb0897c875
Step 12/15 : COPY azureml-setup/spark_cache.py azureml-setup/log4j.properties /azureml-setup/
 ---> 47b099631471
Step 13/15 : RUN if [ $SPARK_HOME ]; then /bin/bash -c '$

{'runId': 'sdk-papermill_1553581647_9f0859a8',
 'target': 'amlcompute',
 'status': 'Completed',
 'startTimeUtc': '2019-03-26T06:45:47.059829Z',
 'endTimeUtc': '2019-03-26T06:55:41.699567Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': '834efa33-b538-4be2-a046-e8f68581b0b9'},
 'runDefinition': {'script': 'papermill_notebook_run_handler.py',
  'arguments': ['-i',
   'notebooks/00_quick_start/sar_movielens.ipynb',
   '-o',
   'outputs/out.ipynb',
   '-e',
   '{"history": true}',
   '-p',
   '{}',
   '-n',
   '{"MOVIELENS_DATA_SIZE": "100k", "TOP_K": 10}'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'amlcompute',
  'dataReferences': {},
  'jobName': None,
  'autoPrepareEnvironment': True,
  'maxRunDurationSeconds': None,
  'nodeCount': 1,
  'environment': {'name': 'Experiment sdk-papermill Environment',
   'version': 2,
   'python': {'interpreterPath': 'python',
    'userManagedDependencies': False,
    'c